In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from scipy import sparse
from joblib import Parallel,delayed
from scipy.signal import savgol_filter
from sklearn.decomposition import PCA
from keras import utils as np_utils
from sklearn.model_selection import train_test_split  
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Dropout,BatchNormalization
from tensorflow.keras import datasets, layers, models
from keras.models import Sequential
from numpy import unique
from sklearn.metrics import mean_squared_error,r2_score
from imblearn.over_sampling import SMOTE
from keras.regularizers import l2
from keras import backend as K

def baseline_als(y, lam, p, niter=10):
    L = len(y)
    D = sparse.diags([1, -2, 1], [0, -1, -2], shape=(L, L - 2))
    w = np.ones(L)
    for i in range(niter):
        W = sparse.spdiags(w, 0, L, L)
        Z = W + lam * D.dot(D.transpose())
        z = spsolve(Z, w * y)
        w = p * (y > z) + (1 - p) * (y < z)
    return z

def root_mean_squared_error(y_val, y_preds):
        return K.sqrt(K.mean(K.square(y_preds - y_val))) 

def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error')
    plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
    plt.ylim([0,1])
    plt.legend()
    plt.show()
                                                                                                     
np.set_printoptions(precision=2)
fnames=sorted(glob('Data_Pathway/*'))
files=[np.loadtxt(f) for f in fnames]
map=[f[:,3] for f in files]
map=[np.reshape(m,(-1,1011)) for m in map]
map_full = np.concatenate([m[:,:] for m in map],axis=0)

y = np.concatenate([np.repeat(i, len(m)) for i, m in enumerate(map)], axis=0)
map_full= savgol_filter(map_full, 11, 3, axis=0)
back = Parallel(n_jobs=8)(delayed(baseline_als)(j, 100000, 0.001) for j in map_full)
map_full= np.subtract(map_full, back)
map_full= np.reshape(map_full, (-1, 1011))
divide=map_full[:,999:1000]
X=map_full/divide

x_train, x_test, y_train, y_test=train_test_split(X,y,test_size=0.2,stratify=y,shuffle=True)
    
pca = PCA(n_components=22)
pca.fit(x_train)

x_train= pca.transform(x_train)

sm = SMOTE()
x_train_res, y_train_res = sm.fit_resample(x_train, y_train)
  
x_train_cnn = np.reshape(x_train_res, (x_train_res.shape[0], x_train_res.shape[1],1))
  
model = Sequential()
model.add(Conv1D(22,7, activation='relu',input_shape=(22,1), strides=1, padding="same"))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv1D(22,7, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv1D(44,7, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv1D(44,7, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(22, activation='relu',kernel_regularizer=l2(0.001)))
model.add(Dense(1, activation='linear'))
model.compile(loss= 'mse' , optimizer='adam', metrics=['mse']) 
model.summary()

history=model.fit(x_train_cnn, y_train_res, epochs=35, batch_size=44,verbose=1)

# evaluate the model
scores = model.evaluate(x_train_cnn, y_train_res, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


x_test= pca.transform(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1],1))

y_preds = model.predict(x_test)

# evaluate the model
score = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
print('Variance score: %.4f' % r2_score(y_test,y_preds))

bars = ('0', '5 fM', '50 fM', '0.5 pM', '5 pM','50 pM','0.5 nM','5 nM','50 nM','0.5 uM','5 uM','50 uM','0.5mM','5 mM')
x_pos = np.arange(len(bars))
plt.xticks(x_pos, bars,rotation ='vertical')
plt.yticks(x_pos, bars)
plt.scatter(y_test,y_preds, s=5, color="blue")
plt.show()

In [ ]:
#Down sample control class  
from imblearn.under_sampling import RandomUnderSampler 
x_test_forunder=np.squeeze(x_test, axis=2)

rus = RandomUnderSampler(random_state=42)
X_rus, y_rus = rus.fit_resample(x_test_forunder, y_test)

x_test_rus = np.reshape(X_rus, (X_rus.shape[0], X_rus.shape[1],1))
y_predict = model.predict(x_test_rus)

r2_score(y_rus,y_predict)
mean_squared_error(y_rus,y_predict)

plt.scatter(y_rus,y_predict, s=5, color="blue")
print(r2_score)
print(mean_squared_error)

plt.show()

In [ ]:
#Plot out figures
import seaborn as sns
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

y_preds_1d=y_predict.flatten()

fig, ax = plt.subplots()

bp=sns.boxplot(x=y_rus,y=y_preds_1d,linewidth=1.3,flierprops = dict(markerfacecolor = 'dodgerblue',markeredgecolor='dodgerblue', markersize = 3,marker='o',),whis=[0.5,99.5])

hfont = {'fontname':'Helvetica'}

labels = ["0","5 fM","5 pM","5 nM","5 \u03bcM","5 mM"]
label = [" ","5 fM","5 pM","5 nM","5 \u03bcM","5 mM"]
ax.tick_params(direction='in', length=8, width=3, colors='black')

ax.xaxis.set_minor_locator(AutoMinorLocator(1))

ax.tick_params(direction='in',which='both', width=1)
ax.tick_params(direction='in',which='minor', length=4)

ax.yaxis.set_minor_locator(AutoMinorLocator(1))

ax.tick_params(which='both', width=1)
ax.tick_params(which='minor', length=4)

ax.set_xticks([0,1,4,7,10,13])
ax.set_yticks([0,1,4,7,10,13])
ax.set_xticklabels(label,fontsize=18,**hfont)
ax.set_yticklabels(labels,fontsize=18,**hfont)
ax.set_xlabel('True Concentrations',fontsize=20,**hfont)
ax.set_ylabel('Predicted Concentrations',fontsize=20,**hfont)

x = np.linspace(0, 14, 13)
plt.plot(x, x + 0, 'black')  # solid green
plt.xlim(-0.5,13.5)
plt.ylim(-0.1,13.5,1)
plt.axhspan(-0.1, 1.8, facecolor='0.9')

plt.show()